## Zadanie 14.2.
#### Napisz program ‘narzedzia.py’ służący do obsługi narzędziowni. Program ma umożliwiać wykonywanie następujących funkcji: dopisanie nowego narzędzia na listę, zmiana liczby sztuk narzędzia na liście (w tym możliwość kompletnego usunięcia narzędzia, gdy liczba sztuk spadnie do zera), wydanie narzędzia (zapamiętanie godziny i nazwiska pracownika biorącego narzędzie), zwrot narzędzia, lista wszystkich narzędzi, lista dostępnych narzędzi, lista wydanych narzędzi. 

(przyjęto założenie, że jeden pracownik może zabrać tylko jedno narzędzie danego typu)

In [1]:
import shelve, sys 
from time import *
from datetime import datetime

In [2]:
#inicjalizacja bazy danych 
def init(): 
    global baza, wszystkie, wydane, dostepne
    try: 
        baza = shelve.open ('baza_narzedziowa', writeback = True) # otwarcie    
    except: 
        print("Błąd krytyczny! Baza danych nie została otwarta!")
        sys.exit(0) # wyjście z programu 
    print("Inicjalizacja udana. Baza danych została otwarta.")
    if 'wszystkie' not in baza.keys():
        baza['wszystkie'] = {}
    if 'wydane' not in baza.keys():
        baza['wydane'] = {}
    if 'dostepne' not in baza.keys():
        baza['dostepne'] = {}
    wszystkie = baza['wszystkie']
    wydane = baza['wydane']
    dostepne = baza['dostepne']

In [3]:
# menu główne programu 
def menu(): 
    while True: 
        print ('\n')
        print ('-'*100) 
        print ('NARZĘDZIOWNIA'.center(100)) 
        print ('-'*100 )
        print ('[+] Dodaj [N] Zmień liczbę [-] Wydaj [Z] Zwróć [A] Wszystkie [D] Dostępne [W] Wydane [K] Koniec'.center(100))
        print ('-'*100) 
        w=input()[0].upper() # pierwszy znak (duza litera) 
        if w in '+N-ZADWK': # znany? 
            return w # tak - zwracamy go 
        print ('Nieznane polecenie -') 

In [4]:
# realizacja wyboru użytkownika 
def wybor(): 
    while True: 
        w=menu() 
        if w=='K': 
            break
        elif w=='+':
            dodaj()
        elif w=='N': 
            zmien_liczbe()
        elif w=='-': 
            wydaj() 
        elif w=='Z': 
            zwroc() 
        elif w=='A': 
            pokaz_wszystkie() 
        elif w=='D':
            pokaz_dostepne()
        elif w=='W':
            pokaz_wydane()

In [5]:
# dodanie narzędzia do listy 
def dodaj(): 
    global wszystkie, dostepne
    print("\nDodanie narzędzia do listy\n")
    try: 
        nazwa=input("Podaj nazwę nowego narzędzia: ").lower()
        ilosc=int(input("Podaj liczbę sztuk nowego narzędzia: ")) 
    except: 
        print("Błąd wprowadzania danych! Narzędzie nie zostało dodane!") 
        return 
    try:
        if nazwa not in wszystkie.keys():
            wszystkie[nazwa] = ilosc
            dostepne[nazwa] = ilosc
            print("Dodano narzędzie",nazwa,"w ilości",ilosc,"sztuk.")
        else:
            print("Takie narzędzie już jest w narzędziowni. Możesz zmienić liczbę sztuk w menu głównym.")
    except: 
        print("Błąd zapisu danych! Narzędzie nie zostało dodane!")

In [6]:
# zmiana liczby sztuk wybranego narzędzia
def zmien_liczbe(): 
    global wszystkie, wydane, dostepne
    print("\nZmiana liczby sztuk narzędzia\n")
    try: 
        nazwa=input("Podaj nazwę narzędzia: ").lower()
        ilosc=int(input("Podaj liczbę sztuk narzędzia: ")) 
    except: 
        print("Błąd wprowadzania danych! Liczba sztuk nie została wczytana!") 
        return 
    try: 
        if nazwa in wszystkie.keys():
            ile_wydanych = sum([1 for element in wydane.keys() if element == nazwa])
            if ilosc >= ile_wydanych:
                wszystkie[nazwa] = ilosc
                dostepne[nazwa] = ilosc - ile_wydanych
                if dostepne[nazwa] == 0:
                    usun = input("Aktualna liczba dostępnych sztuk tego narzędzia to 0. Czy chcesz usunąć je całkowicie z bazy dostępnych narzędzi? [T/N]")
                    if usun == 'T':
                        del dostepne[nazwa]
                    elif usun == 'N':
                        print("Narzędzie nie zostało usunięte z listy dostępnych narzędzi.")
                    else:
                        print("Nieznane polecenie. Narzędzie nie zostało usunięte z listy.")
            else:
                print("Nie wprowadzono zmiany. Narzędzi musi być co najmniej tyle, ile wydanych sztuk!")
        else:
            print("Nie ma takiego narzędzia. Liczba sztuk nie została zmieniona!")
    except: 
        print("Błąd zapisu danych! Liczba sztuk nie została zmieniona!")

In [7]:
# wydanie narzędzia
def wydaj(): 
    global wszystkie, wydane, dostepne
    print("\nWydanie narzędzia\n")
    try: 
        nazwa=input("Podaj nazwę narzędzia: ").lower()
        nazwisko=input("Podaj imię i nazwisko wypożyczającego: ")
    except: 
        print("Błąd wprowadzania danych! Dane nie zostały wczytane!") 
        return 
    try: 
        godzina = datetime.now()
        if sum([1 for narz, attr in wydane.items() if narz == nazwa and attr['nazwisko'] == nazwisko]) == 0:
            if nazwa in dostepne.keys():
                wydane[nazwa] = {}
                wydane[nazwa]['nazwisko'] = nazwisko
                wydane[nazwa]['godzina'] = godzina
                dostepne[nazwa] -= 1
                if dostepne[nazwa] == 0:
                    usun = input("Aktualna liczba dostępnych sztuk tego narzędzia to 0. Czy chcesz usunąć je całkowicie z bazy dostępnych narzędzi? [T/N]")
                    if usun == 'T':
                        del dostepne[nazwa]
                    elif usun == 'N':
                        print("Narzędzie nie zostało usunięte z listy dostępnych narzędzi.")
                    else:
                        print("Nieznane polecenie. Narzędzie nie zostało usunięte z listy.")
                print("Wydano narzędzie.")
            else:
                print("Narzędzie nie jest dostępne. Nie można go wydać!")
        else:
            print("Ten pracownik wypożyczył już",nazwa,'.')
    except: 
        print("Błąd zapisu danych! Nie wydano narzędzia!")

In [8]:
# zwrot narzędzia
def zwroc(): 
    global wydane, dostepne
    print("\nZwrot narzędzia\n")
    try: 
        nazwa=input("Podaj nazwę narzędzia: ").lower()
        nazwisko=input("Podaj imię i nazwisko osoby, która wypożyczyła narzędzie: ")
    except: 
        print("Błąd wprowadzania danych! Dane nie zostały wczytane!") 
        return 
    try: 
        if nazwa in wydane.keys():
            for narz, attr in wydane.items():
                if narz == nazwa and attr['nazwisko'] == nazwisko:
                    if nazwa in dostepne.keys():
                        dostepne[nazwa] += 1
                    else:
                        dostepne[nazwa] = 1
                    do_usuniecia = narz
                    print("Narzędzie zostało zwrócone, dziękujemy.")
                else:
                    print("To narzędzie wypożyczyła inna osoba. Zwrot nie jest możliwy.")
            del wydane[do_usuniecia]
        else:
            print("Nie wydano takiego narzędzia. Zwrot nie jest możliwy.")
    except: 
        print("Błąd zapisu danych! Narzędzie nie zostało zwrócone!")

In [9]:
# lista wszystkich narzędzi
def pokaz_wszystkie(): 
    global wszystkie
    print ()
    print ('Lista wszystkich narzędzi'.center(38)) 
    print ('-'*38) 
    print ('|'+'Nazwa'.center(20)+'|'+'Liczba sztuk'.center(15)+'|' )
    print( '-'*38 )
    if len(wszystkie):
        for narz, ilosc in wszystkie.items():
            print ("|%20s|%15s|" % (narz.center(20), str(ilosc).center(15)) )
            print( '-'*38)
    else:
        print('|'+"Brak narzędzi.".center(36)+'|')
        print( '-'*38) 

In [10]:
# lista dostępnych narzędzi
def pokaz_dostepne(): 
    global dostepne
    print ('Lista dostępnych narzędzi'.center(38)) 
    print ('-'*38) 
    print ('|'+'Nazwa'.center(20)+'|'+'Liczba sztuk'.center(15)+'|' )
    print( '-'*38 )
    if len(dostepne.items()):
        for narz, ilosc in dostepne.items():
            print ("|%20s|%15s|" % (narz.center(20), str(ilosc).center(15)) )
            print( '-'*38)
    else:
        print('|'+"Brak narzędzi.".center(36)+'|')
        print( '-'*38) 

In [11]:
# lista wydanych narzędzi
def pokaz_wydane(): 
    global wydane 
    print ()
    print ('Lista wydanych narzędzi'.center(65)) 
    print ('-'*65) 
    print ('|'+'Nazwa'.center(20)+'|'+'Imię i nazwisko'.center(20)+'|'+'Godzina'.center(21)+'|')
    print( '-'*65 )
    if len(wydane.keys()):
        for narz, attr in wydane.items():
            print ("|%20s|%20s|%21s|" % (narz.center(20), attr['nazwisko'].center(20), attr['godzina'].strftime("%H:%M (%Y-%m-%d)").center(21)) )
            print( '-'*65)
    else:
        print("|"+'Brak narzędzi'.center(63)+"|")
        print( '-'*65) 

In [12]:
# program główny 
init() # otwarcie bazy 
try: 
     wybor() # interfejs użytkownika 
except: 
     print ("Wystąpił poważny błąd." )
baza.close() # zamknięcie bazy 

Inicjalizacja udana. Baza danych została otwarta.


----------------------------------------------------------------------------------------------------
                                           NARZĘDZIOWNIA                                            
----------------------------------------------------------------------------------------------------
  [+] Dodaj [N] Zmień liczbę [-] Wydaj [Z] Zwróć [A] Wszystkie [D] Dostępne [W] Wydane [K] Koniec   
----------------------------------------------------------------------------------------------------
+

Dodanie narzędzia do listy

Podaj nazwę nowego narzędzia: grabie
Podaj liczbę sztuk nowego narzędzia: 2
Dodano narzędzie grabie w ilości 2 sztuk.


----------------------------------------------------------------------------------------------------
                                           NARZĘDZIOWNIA                                            
------------------------------------------------------------------------------------------